In [2]:
import pandas as pd
import spacy
import re
import csv
import json
import ast
import string

# Excel Section może się pojawi

# CSV SECTION

*Koncept*

Plan jest taki żeby przetworzyć dane tak by pasowały do pliku konkursowego. Z racji że dane które znalazłem były podzielone w dość specyficzny sposób, zawierały jedną kolumnę `text` 

Przykład wiersza

`<s>[INST] I need a summary of the book Existential Therapy: 100 Key Points and Techniques for Van Kilback. Can you help? [/INST] I need a summary of the book Existential Therapy: 100 Key Points and Techniques for [FULLNAME]. Can you help? </s>`

stwierdziłem że podziele ję na 2 kolumny `full_text` zgodną z konkursem i tymczasową kolumnę `tagged_text` która posłuży do stworzenia kolumny labels będzie to tekst który zawiera tag(tekst po `[/INST]`)

### Wczytanie danych i stworzenie pliku z tagged_text

In [44]:
# Wczytaj dane z pliku CSV
csv_data_tagged = pd.read_csv('PII-Redaction.csv') 

def extract_text_after_tag(text):
    """funkcja wyodrębnia tekst po tagu [/INST]"""
    if '[/INST]' in text:
        split_text = text.split('[/INST]')
        extracted_text = split_text[1].strip() if len(split_text) > 1 else None
        return extracted_text.replace('</s>', '').strip() if extracted_text else None
    else:
        return text.strip().replace('</s>', '')



# Wyrażenie regularne do wydobycia żądanej części tekstu
pattern = r"\[INST\](.*?)\[/INST\]"
def extract_text(text):
    """wyodrębnienia żądanej części tekstu z każdego wiersza"""
    matches = re.findall(pattern, text)
    if matches:
        return matches[0].strip()  # Zwracamy pierwsze dopasowanie, usuwając ewentualne białe znaki na początku i końcu
    else:
        return None

# Wydobycie tekstu z każdego wiersza do kolumny full_text
csv_data_tagged['full_text'] = csv_data_tagged['text'].apply(extract_text)

# Wydobycie tekstu z każdego wiersza  ztagami do kolumny tagged_text
csv_data_tagged['tagged_text'] = csv_data_tagged['text'].apply(extract_text_after_tag)

# nazwy kolumn, które chcemy zachować
column_to_keep = ['full_text', 'tagged_text']

# Zachowanie tylko wybranej kolumny
csv_data_tagged = csv_data_tagged.loc[:, column_to_keep]

# Zapisanie zmodyfikowanych danych do pliku CSV - kolumna full_text z usuniętymi oznaczeniami
csv_data_tagged.to_csv('full_tag_csv.csv', index=False)


### Standaryzacja tekstu w full_text i tagged_text

In [45]:
# nie działa tak jak planowałem i nie do końca wiem dlaczego w sensie nie zamienia \ na \n\n
def standardize_tagged_csv(input_file, output_file):
    # Wczytaj plik CSV
    data = pd.read_csv(input_file)

    # Ustandaryzuj każdy wiersz
    data['full_text'] = data['full_text'].apply(lambda x: '"' + re.sub(r'^\d+\.\s*', '', x.strip('""').strip()) + '"' if not x.startswith('"') else x)
    data['tagged_text'] = data['tagged_text'].apply(lambda x: '"' + re.sub(r'^\d+\.\s*', '', x.strip('""').strip()) + '"' if not x.startswith('"') else x)
    # Zapisz do pliku CSV w taki sposób żeby nie było w """"""
    data.to_csv(output_file, index=False, quoting=csv.QUOTE_NONE, escapechar='\n')

# Przykładowe użycie funkcji
standardize_tagged_csv('full_tag_csv.csv', 'full_tag_std_csv.csv')

### dodanie kolumn, document, tokens, trailing_whitespace

In [46]:
# Wczytanie danych z pliku CSV
csv_data_tagged = pd.read_csv('full_tag_std_csv.csv')

# Dodanie kolumny 'document'
csv_data_tagged['document'] = range(1, len(csv_data_tagged) + 1)

# Wczytanie modelu języka angielskiego
nlp = spacy.load('en_core_web_sm')

# Funkcja do tokenizacji tekstu i sprawdzenia, czy token kończy się białym znakiem
# def tokenize_and_check_whitespace(text):
#     doc = nlp(text)
#     tokens = [token.text for token in doc]
#     trailing_whitespace = [token.whitespace_ != '' for token in doc]
#     return tokens, trailing_whitespace
def tokenize_and_check_whitespace(text):
    """Tokenizacja tekstu i sprawdzenie, czy token kończy się białym znakiem."""
    doc = nlp(text)  # Tokenizacja tekstu
    tokens = []  # Lista tokenów
    trailing_whitespace = []  # Lista informacji o białym znaku na końcu
    
    for token in doc:
        # Pomijanie interpunkcji
        if token.text not in string.punctuation:
            tokens.append(token.text)
            trailing_whitespace.append(token.whitespace_ != '')

    return tokens, trailing_whitespace


# Dodanie kolumn 'tokens' i 'trailing_whitespace'
csv_data_tagged[['tokens', 'trailing_whitespace']] = csv_data_tagged['full_text'].apply(tokenize_and_check_whitespace).apply(pd.Series)

csv_data_tagged = csv_data_tagged[['document','full_text','tokens', 'trailing_whitespace', 'tagged_text']]



csv_data_tagged.to_csv('csv_tagged_with_added_columns.csv', index=False)
# Zapisanie DataFrame jako plik JSON
csv_data_tagged.to_json('csv_tagged.json')

### Dodanie labels do pliku z kolumną z tagami

In [64]:
def assign_labels(tagged_text):
    """funkcja do tworzenia listy labeli zgodnie z opisem konkursu, robi to na podstaw"""
    words = tagged_text.split()
    labels = []
    for i, word in enumerate(words):
        if '[' in word:
            # Wyciągamy tag bez dodatkowych znaków
            tag = word.split('[')[-1].split(']')[0].upper()
            if tag == 'FIRSTNAME':
                # Sprawdzamy, czy kolejne tagi po pierwszym tagu imienia to nazwisko
                next_word_index = i + 1
                if next_word_index < len(words) and '[LASTNAME]' in words[next_word_index]:
                    labels.append("B-NAME_STUDENT")
                    labels.append("I-NAME_STUDENT")
                else:
                    labels.append("B-NAME_STUDENT")
            elif tag == 'LASTNAME':
                # Sprawdzamy, czy poprzedni tag to imię
                prev_word_index = i - 1
                if prev_word_index >= 0 and '[FIRSTNAME]' in words[prev_word_index]:
                    continue  # Pomijamy przypisanie etykiety w tym przypadku
                else:
                    labels.append("B-NAME_STUDENT")
            elif tag == 'FULLNAME':
                # Sprawdzamy, czy lista labels ma co najmniej dwa elementy
                    labels.append("B-NAME_STUDENT")
                    labels.append("I-NAME_STUDENT") 
            elif tag == 'EMAIL':    
                labels.append("B-EMAIL")
            elif tag in ['USERNAME', 'DISPLAYNAME']:
                labels.append("B-USERNAME")
            else:
                labels.append('O')
        else:
            labels.append('O')
    return labels



# Wczytanie danych z pliku CSV
csv_data = pd.read_csv('csv_tagged_with_added_columns.csv') 

# Dodanie kolumny z etykietami
csv_data['labels'] = csv_data['tagged_text'].apply(assign_labels)

# Zapisanie zmodyfikowanych danych do pliku CSV
csv_data.to_csv('csv_tagged_with_labels.csv', index=False)

### Usunięcie tych wierszy w których liczba słów w tokens, trailing_whitespace, labels nie zgadza się

In [5]:
csv_data = pd.read_csv('csv_tagged_with_labels.csv')

# Przekształć 'tokens', 'trailing_whitespace' i 'labels' na listy
csv_data['tokens'] = csv_data['tokens'].apply(ast.literal_eval)
csv_data['trailing_whitespace'] = csv_data['trailing_whitespace'].apply(ast.literal_eval)
csv_data['labels'] = csv_data['labels'].apply(ast.literal_eval)

# Sprawdź, czy liczba tokenów, etykiet i informacji o białym znaku na końcu jest taka sama dla każdego wiersza
csv_data['num_tokens'] = csv_data['tokens'].apply(len)
csv_data['num_labels'] = csv_data['labels'].apply(len)
csv_data['num_trailing_whitespace'] = csv_data['trailing_whitespace'].apply(len)

# Usuń wiersze, w których liczby nie są sobie równe
csv_data = csv_data[(csv_data['num_tokens'] == csv_data['num_labels']) & (csv_data['num_tokens'] == csv_data['num_trailing_whitespace'])]

# Usuwanie zbędnych kolumn
csv_data = csv_data.drop(columns=['num_tokens', 'num_labels', 'num_trailing_whitespace', 'tagged_text'])
# aktualizacja numeracji
csv_data['document'] = range(1, len(csv_data) + 1)
# kolejność
csv_data = csv_data[['document','full_text','tokens', 'trailing_whitespace', 'labels']]
# Zapisz zmodyfikowane dane
csv_data.to_csv('csv_tagged_with_labels_filtered.csv', index=False)
# zapis do json
csv_data.to_json('final_csv.json', orient='records')

### usunięcie labels: null ze zbioru

In [10]:
# Wczytanie danych z pliku JSON
data = pd.read_json('final_csv.json')

# Usunięcie wierszy, w których kolumna "labels" ma wartość null
data = data.dropna(subset=['labels'])

# Zapisanie zmodyfikowanych danych do pliku JSON
data.to_json('final_csv.json', orient='records')

## Łączenie plików 

#### Wczytanie pliku json, sortowanie od największego do najmniejszego

In [11]:
train_data = pd.read_json('train.json')
train_data.sort_values('document', ascending=False)

,document,full_text,tokens,trailing_whitespace,labels
6806,22687,Mind Mapping\n\nChallenge\n\nMy consulting tea...,"[Mind, Mapping, \n\n, Challenge, \n\n, My, con...","[True, False, False, False, False, True, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6805,22684,Brainstorming\n\nChallenge & Selection\n\nBrai...,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6804,22681,"Challenge\n\nSo, a few months back, I had chos...","[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6803,22679,Why Mind Mapping?\n\nMind maps are graphical r...,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6802,22678,EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE My w...,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...,...,...
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."


#### połączenie danych zmiana numeracji

In [12]:
# Wczytanie danych z plików JSON
train_data = pd.read_json('train.json')
csv_data = pd.read_json('final_csv.json')
# excel_data = pd.read_json('excel.json')

# Ustalenie początkowego numeru dokumentu dla danych z plików excel.json i csv.json
next_document_number = train_data['document'].max() + 1

# Aktualizacja numerów dokumentów w danych z plików excel.json i csv.json
# excel_data['document'] = range(next_document_number, next_document_number + len(excel_data))
# csv_data['document'] = range(next_document_number + len(excel_data), next_document_number + len(excel_data) + len(csv_data))
csv_data['document'] = range(next_document_number, next_document_number + len(csv_data))

# Dodanie danych z plików excel.json i csv.json do danych z pliku train.json
# train_data = pd.concat([train_data, excel_data, csv_data], ignore_index=True)
train_data = pd.concat([train_data, csv_data], ignore_index=True)

# Zapisanie połączonych danych do pliku train.json
train_data.to_json('new_train.json', orient='records')
